# DL model embedding (PYTORCH)

In [ ]:
# !pip install torch

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot

In [15]:
vocab_size = 5000
one_hot_object = [one_hot(words,vocab_size) for words in X_data]
one_hot_object


embedded_words = pad_sequences(
    one_hot_object,
    padding="post",
    maxlen = 25
)
embedded_words

array([[4027, 2561, 1473, ...,    0,    0,    0],
       [2996, 3834, 2194, ...,    0,    0,    0],
       [2073, 4066, 2685, ...,    0,    0,    0],
       ...,
       [2180, 3485, 4315, ...,    0,    0,    0],
       [  87, 4374,  405, ...,    0,    0,    0],
       [4695, 3902, 2428, ...,    0,    0,    0]], dtype=int32)

In [16]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(embedded_words,Y_data,random_state=42,test_size=0.15)


X_test = torch.tensor(X_test)
Y_test = torch.FloatTensor(Y_test.reshape(-1,1))


from torch.utils.data import TensorDataset,DataLoader
train_dt = TensorDataset(torch.tensor(X_train),torch.FloatTensor(Y_train.reshape(-1,1)))
train_loader = DataLoader(train_dt,batch_size=16)


In [19]:

class BRNN(nn.Module):
  def __init__(self):
    super().__init__()
    self.emb = nn.Embedding(num_embeddings=10000,embedding_dim=40)
    self.r1 = nn.LSTM(40,hidden_size = 180,batch_first=True,num_layers=2,bidirectional=True)
    self.l1 = nn.Linear(in_features=180,out_features=140)
    self.l2 = nn.Linear(in_features=140,out_features=120)
    self.l3 = nn.Linear(in_features=120,out_features=42)
    self.out = nn.Linear(in_features=42,out_features=1)

  def Forward(self,x):
    net = self.emb(x)
    out_ ,(ht,c) = self.r1(net)
    net = self.l1(ht[-1])
    net = F.relu(net)
    net = self.l2(net)
    net = F.relu(net)
    net = self.l3(net)
    net = F.relu(net)
    net = self.out(net)
    out = F.sigmoid(net)
    return out

In [20]:
#training the model


def train(model,epochs):
  model_ = model
  model_.zero_grad()
  model_.train()
  optimizer = torch.optim.RMSprop(model_.parameters(),lr=0.001)
  loss_func = nn.BCELoss()
  loss = []
  for epoch in range(1,epochs+1):
    for feature,label in train_loader:
      Y_pred = model.Forward(feature)
      loss_ = loss_func(Y_pred,label)
      optimizer.zero_grad()
      loss_.backward()
      optimizer.step()
    loss.append(loss_.item())
    if epoch:
      print("epoch :",epoch)
      print("loss" ,loss_.item())
      print("=="*30,"\n")
  return loss

def evaluate(model,data):
  with torch.no_grad():
    model_ = model
    model_.eval()
    predictions = []
    for i in data:
      y = model_.Forward(i.reshape(1,-1))
      if y.item() >= 0.5:
        y = 1
        predictions.append(y)
      else:
        y = 0
        predictions.append(y)
  return predictions

In [21]:
loss = train(BRNN(),10)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


epoch : 1
loss 0.3737410008907318

epoch : 2
loss 0.36115115880966187

epoch : 3
loss 0.43766701221466064

epoch : 4
loss 0.3280957341194153

epoch : 5
loss 0.2526642382144928

epoch : 6
loss 0.10852375626564026

epoch : 7
loss 0.02371389977633953

epoch : 8
loss 0.12490575760602951

epoch : 9
loss 0.016773074865341187

epoch : 10
loss 0.07747146487236023



In [22]:
predictions = evaluate(BRNN(),X_test)


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [23]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_test,predictions)

0.5067391077212412

In [ ]:
torch.save(BRNN(),"Sentiment_pytorch_model")

observation---> Pytorch long short term memory seq to veq model has underperformed hence we will select either tfidf or BOW model